In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Define the path to the data directory
data_path = './data' # Or wherever you unzipped the files

# Load the datasets
print("Loading datasets...")
aisles_df = pd.read_csv(os.path.join(data_path, 'aisles.csv'))
departments_df = pd.read_csv(os.path.join(data_path, 'departments.csv'))
orders_df = pd.read_csv(os.path.join(data_path, 'orders.csv'))
products_df = pd.read_csv(os.path.join(data_path, 'products.csv'))
order_products_prior_df = pd.read_csv(os.path.join(data_path, 'order_products__prior.csv'))
order_products_train_df = pd.read_csv(os.path.join(data_path, 'order_products__train.csv'))
print("Datasets loaded.")

# --- Initial Exploration ---

print("\n--- Aisles Info ---")
aisles_df.info()
print(aisles_df.head())

print("\n--- Departments Info ---")
departments_df.info()
print(departments_df.head())

print("\n--- Orders Info ---")
orders_df.info()
print(orders_df.head())
# Check missing values specifically for days_since_prior_order (expected for first orders)
print(f"Missing days_since_prior_order: {orders_df['days_since_prior_order'].isnull().sum()}")
# Check how many orders belong to train/test/prior sets
print(f"Order eval_set distribution:\n{orders_df['eval_set'].value_counts()}")


print("\n--- Products Info ---")
products_df.info()
print(products_df.head())

print("\n--- Order Products Prior Info ---")
order_products_prior_df.info() # This might take a moment, it's large
print(order_products_prior_df.head())

print("\n--- Order Products Train Info ---")
order_products_train_df.info()
print(order_products_train_df.head())

Loading datasets...
Datasets loaded.

--- Aisles Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation

--- Departments Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 468.0+ bytes
   department_id department
0            

In [5]:
# --- Target Variable Creation (Predict New Product Purchase) ---

# Step 1: Identify Users with a 'Train' Order
print("\nStep 1: Identifying Train Users...")
train_user_ids = orders_df[orders_df['eval_set'] == 'train']['user_id'].unique()
print(f"Number of users in the training set: {len(train_user_ids)}")

# Get the specific order_id for each user's train order
train_orders = orders_df[(orders_df['user_id'].isin(train_user_ids)) & (orders_df['eval_set'] == 'train')]
print(f"Number of 'train' orders: {len(train_orders)}")
# print(train_orders.head()) # Keep output concise for now

# Step 2: Get Products in Each 'Train' Order
print("\nStep 2: Getting Products in Train Orders...")
# Merge train orders with their products
train_order_details = pd.merge(train_orders[['order_id', 'user_id']], order_products_train_df[['order_id', 'product_id']], on='order_id', how='left')
# Handle potential empty train orders (though unlikely in this dataset)
train_order_details['product_id'] = train_order_details['product_id'].fillna(-1) # Use -1 or similar placeholder if needed

print(f"Number of rows in train_order_details: {len(train_order_details)}")

# Create a set of products for each train order_id for quick lookup
# Group by user_id directly as each train user has only one train order
train_products_by_user = train_order_details.groupby('user_id')['product_id'].apply(set).reset_index()
train_products_by_user.rename(columns={'product_id': 'train_product_set'}, inplace=True)

print(f"Number of users mapped to train product sets: {len(train_products_by_user)}")
# print("\nProducts sets for train users (first few):")
# print(train_products_by_user.head())

# Step 3: Get Historical (Prior) Products for Each Train User
print("\nStep 3: Getting Prior Products for Train Users...")
# Get all 'prior' orders for the users who are in the train set
prior_orders_for_train_users = orders_df[(orders_df['user_id'].isin(train_user_ids)) & (orders_df['eval_set'] == 'prior')]
print(f"Number of prior orders from train users: {len(prior_orders_for_train_users)}")

# Merge with prior order products
# Select only necessary columns for efficiency
prior_order_details = pd.merge(
    prior_orders_for_train_users[['order_id', 'user_id']],
    order_products_prior_df[['order_id', 'product_id']],
    on='order_id',
    how='inner'
)
print(f"Total prior order-product entries for train users: {len(prior_order_details)}")

# Get the set of unique products ever purchased by each user in their prior history
# This can be memory intensive, do it efficiently
user_prior_products = prior_order_details.groupby('user_id')['product_id'].apply(set).reset_index()
user_prior_products.rename(columns={'product_id': 'prior_product_set'}, inplace=True)

# Free up memory from the large intermediate dataframe
del prior_order_details
del prior_orders_for_train_users
import gc
gc.collect()

print(f"Number of users mapped to prior product sets: {len(user_prior_products)}")
# print("\nUnique historical products per user (first few):")
# print(user_prior_products.head())

# Step 4: Create the Target Variable y
print("\nStep 4: Creating the Target Variable...")
# Merge the user's train product set and prior product set
# Use train_products_by_user which already has user_id and train_product_set
user_target_data = pd.merge(train_products_by_user, user_prior_products, on='user_id', how='left')

# Handle cases where a user might *only* have a 'train' order (no prior history)
# These users definitely bought 'new' products relative to their (empty) history.
# The 'left' merge already handled users present in train_products_by_user but not user_prior_products
# Their 'prior_product_set' will be NaN. We'll fillna with empty sets.
user_target_data['prior_product_set'] = user_target_data['prior_product_set'].apply(lambda x: x if isinstance(x, set) else set())

print(f"Number of users in target data: {len(user_target_data)}")
# print("\nUser data with train and prior product sets (first few):")
# print(user_target_data.head())

# Define the function to check for new products
def check_for_new_products(row):
    train_set = row['train_product_set']
    prior_set = row['prior_product_set']
    # Handle the placeholder if we used one for empty train orders
    if -1 in train_set:
        train_set.remove(-1)
        if not train_set: # If -1 was the only thing, the order was empty
             return 0 # Or decide how to handle this edge case

    # Calculate the difference: products in train_set but not in prior_set
    new_products = train_set.difference(prior_set)
    # Return 1 if there's at least one new product, 0 otherwise
    return 1 if len(new_products) > 0 else 0

# Apply the function to create the target column 'new_product_purchased'
print("Applying function to calculate target...")
user_target_data['new_product_purchased'] = user_target_data.apply(check_for_new_products, axis=1)

# Select the columns we need for the target definition: user_id and the target
final_target = user_target_data[['user_id', 'new_product_purchased']].copy()

# Free up memory
del user_target_data
del train_products_by_user
del user_prior_products
gc.collect()

print("\nFinal target variable per user (first few):")
print(final_target.head())

# Check the distribution of the target variable
print("\nTarget variable distribution:")
target_distribution = final_target['new_product_purchased'].value_counts(normalize=True) * 100
print(target_distribution)


Step 1: Identifying Train Users...
Number of users in the training set: 131209
Number of 'train' orders: 131209

Step 2: Getting Products in Train Orders...
Number of rows in train_order_details: 1384617
Number of users mapped to train product sets: 131209

Step 3: Getting Prior Products for Train Users...
Number of prior orders from train users: 2047377
Total prior order-product entries for train users: 20641991
Number of users mapped to prior product sets: 131209

Step 4: Creating the Target Variable...
Number of users in target data: 131209
Applying function to calculate target...

Final target variable per user (first few):
   user_id  new_product_purchased
0        1                      1
1        2                      1
2        5                      1
3        7                      1
4        8                      1

Target variable distribution:
new_product_purchased
1    81.555381
0    18.444619
Name: proportion, dtype: float64
